In [1]:
import sys
import os
import numpy as np
import multiprocessing
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
import time
from scipy.stats import norm
import datetime as dt

module_path = os.path.abspath(os.path.join('.'))
sys.path.append(module_path + "/../../notebooks/vax_sims/")
from vax_sims_LHS_samples import *
from plot_utils import *
sys.path.append(module_path + "/../../src/simulations_v2")
from stochastic_simulation import StochasticSimulation

In [2]:
UNCERTAINTY_PARAMS = ['vax_susc_mult', 'vax_transmission_mult', 'contacts_per_day_mult', 'outside_infection_rate_mult',
                      'cases_isolated_per_contact_trace', 'initial_ID_prevalence']

UNCERTAINTY_PARAM_RANGES = {
    'vax_susc_mult': (0.097608, 0.941192), # 0.5194 +/- 1.96 * 0.2152
    'vax_transmission_mult': (0.25, 1),
    'contacts_per_day_mult': (0.9,2.7),
    'outside_infection_rate_mult': (1, 5),
    'cases_isolated_per_contact_trace': (0.5,1.5),
    'initial_ID_prevalence': (0.003, 0.0054)
}

def load_posterior_df():
    df = pd.read_csv('../vax_sims/posterior_csvs/21_10_08_14:44_posteriors.csv')
    return df

df = load_posterior_df()
MLE_point = list(df.sort_values('posterior', ascending=False).head(1)[UNCERTAINTY_PARAMS].iloc[0])

In [3]:
group_params = load_params("../vax_sims/vax_sim_nominal_params/ug_greek_athlete_nominal.yaml")[1]

vax_susc_mult = MLE_point[0]
vax_trans_mult = MLE_point[1]

# Contacts per day should be 7.59 (7.36 + 0.23) from the contact matrix
omicron_multiplier = 5
social_distance_multiplier = 1
group_params['expected_contacts_per_day'] = 7.59 * vax_susc_mult * vax_trans_mult * MLE_point[2] * omicron_multiplier * social_distance_multiplier

group_params['daily_outside_infection_p'] = group_params['daily_outside_infection_p'] * MLE_point[3] * 10
group_params['cases_isolated_per_contact'] *= MLE_point[4]
group_params['initial_ID_prevalence'] = 0.04

# 66% of (3533+8434) ~= 8018
group_params['population_size'] = 8018
group_params['test_population_fraction'] = 1/7

/home/aaj54/group-testing/src/simulations_v2/load_params.py:136: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  params = yaml.load(f)
/home/aaj54/group-testing/src/simulations_v2/load_params.py:45: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  age_sev_params = yaml.load(f)


In [4]:
sim = StochasticSimulation(group_params)

In [5]:
sim_results = list()
for _ in range(100):
    sim_results.append(sim.run_new_trajectory(35))

In [6]:
sim_results[0]

,S,QS,QI,R,E_0,E_1,E_2,E_3,E_4,E_5,...,SyID_severe_17,SyID_severe_18,SyID_severe_19,cumulative_mild,cumulative_severe,cumulative_outside_infections,severity_0,severity_1,severity_2,severity_3
0,7697,0,0,0,0,0,0,0,0,0,...,1,0,0,11,9,0,11,8.77864,0.162142,0.0592204
1,7151,0,30,0,143,145,99,47,17,10,...,1,0,0,44,37,4,44,36.09,0.666585,0.243462
2,6536,0,100,9,319,256,133,64,25,4,...,0,0,0,118,119,5,118,116.073,2.14388,0.783026
3,5726,0,190,25,486,338,207,79,32,6,...,1,1,2,239,213,8,239,207.761,3.83737,1.40155
4,4477,0,344,58,667,541,266,119,52,17,...,3,3,2,404,363,10,404,354.072,6.53974,2.38856
5,2929,0,614,106,959,676,320,158,58,14,...,8,4,0,637,660,11,637,643.767,11.8904,4.34283
6,1267,0,974,181,1121,776,335,145,69,19,...,6,3,5,981,1044,11,981,1018.32,18.8085,6.86957
7,212,0,1536,326,1059,543,196,88,32,9,...,14,8,6,1489,1627,11,1489,1586.98,29.3117,10.7057
8,0,0,2346,508,451,49,40,15,11,2,...,14,10,5,2180,2357,11,2180,2299.03,42.4633,15.5092
9,0,0,3273,766,0,0,0,0,0,0,...,8,5,6,2960,3143,11,2960,3065.7,56.6237,20.6811


In [7]:
pickle.dump(sim_results, open('UG_virtual_sim_results.p', 'wb'))